# <center> Передача своей целевой функции в модель LGBM 

## Example data

In [20]:
import numpy as np
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error

df_train = pd.read_csv('regression.train', header=None, sep='\t')
df_test = pd.read_csv('regression.test', header=None, sep='\t')


## Objective and Loss

<b>fobj</b> (callable or None, optional (default=None)) –

Customized objective function. Should accept two parameters: <b>preds</b>, <b>train_data</b>, and return (grad, hess).

predslist or numpy 1-D array
The predicted values.

train_dataDataset
The training dataset.

gradlist or numpy 1-D array
The value of the first order derivative (gradient) for each sample point.

hesslist or numpy 1-D array
The value of the second order derivative (Hessian) for each sample point.

For binary task, the preds is margin. For multi-class task, the preds is group by class_id first, then group by row_id. If you want to get i-th row preds in j-th class, the access way is score [j * num_data + i] and you should group grad and hess in this way as well.

In [41]:
def my_logistic_obj(y_hat, dtrain):
    y = dtrain.get_label()
    p = y_hat 
    grad = 4 * p * y + p - 5 * y
    hess = (4 * y + 1) * (p * (1.0 - p))
    return grad, hess

<b>feval</b> (callable or None, optional (default=None)) –

Customized evaluation function. Should accept two parameters: <b>preds</b>, <b>train_data</b>, and return (eval_name, eval_result, is_higher_better) or list of such tuples.

predslist or numpy 1-D array
The predicted values.

train_dataDataset
The training dataset.

eval_namestring
The name of evaluation function (without whitespaces).

eval_resultfloat
The eval result.

is_higher_betterbool
Is eval result higher better, e.g. AUC is is_higher_better.

For binary task, the preds is probability of positive class (or margin in case of specified fobj). For multi-class task, the preds is group by class_id first, then group by row_id. If you want to get i-th row preds in j-th class, the access way is preds[j * num_data + i]. To ignore the default metric corresponding to the used objective, set the metric parameter to the string "None" in params.

In [45]:
def my_err_rate(y_hat, dtrain):
    y = dtrain.get_label()
    y_hat = np.clip(y_hat, 10e-7, 1-10e-7)
    loss_fn = y*np.log(y_hat)
    loss_fp = (1.0 - y)*np.log(1.0 - y_hat)
    return 'myloss', np.sum(-(5*loss_fn+loss_fp))/len(y), False

In [42]:
y_train = df_train[0]
y_test = df_test[0]
X_train = df_train.drop(0, axis=1)
X_test = df_test.drop(0, axis=1)

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

params = {
    'boosting_type': 'gbdt',
    'objective': 'None', #'None'
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [44]:
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=40,
                valid_sets=lgb_eval,
                early_stopping_rounds=5,
                fobj = my_logistic_obj,
                feval = my_err_rate)

gbm.save_model('model.txt')

# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

[1]	valid_0's l2: 0.544	valid_0's l1: 0.544	valid_0's myloss: 37.5782
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l2: 0.544	valid_0's l1: 0.544	valid_0's myloss: 37.5782
[3]	valid_0's l2: 0.544	valid_0's l1: 0.544	valid_0's myloss: 37.5782
[4]	valid_0's l2: 0.544	valid_0's l1: 0.544	valid_0's myloss: 37.5782
[5]	valid_0's l2: 0.544	valid_0's l1: 0.544	valid_0's myloss: 37.5782
[6]	valid_0's l2: 0.544	valid_0's l1: 0.544	valid_0's myloss: 37.5782
Early stopping, best iteration is:
[1]	valid_0's l2: 0.544	valid_0's l1: 0.544	valid_0's myloss: 37.5782
The rmse of prediction is: 0.737563556583431


## Loss for Ion Switching

Хотим сделать больший вес последним наблюдениям.

$$ y,\hat{y} \in \{0, \dots, 9\} \\ $$
$$ Loss_1(y, \hat{y}) = \frac{1}{n} \sum_{i=0}^{n} (y_i = \hat{y}_i ) (y_i + 1) \\ $$
$$ Loss_2(y, \hat{y}) = \sum_{k=0}^{10} (k + 1) f_1(y, \hat{y}) |\quad  y = k \\ $$

Эти функции потерь отражают то, что мы хотим, но они не нормированны.

$$ Loss_3(y, \hat{y}) = \frac{(t + 2)(t +1)}{2}\sum_{k=0}^{t} (k + 1) f_1(y, \hat{y}) |\quad  y = k \\ $$

У этой функции есть проблемма: её долго считать.